In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import initializers
from preprocessing import parse_aug_fn, parse_fn

In [3]:
valid_data = tfds.load('cifar10', split='train[:10%]')

train_data, info = tfds.load('cifar10', split='train[10%:100%]', with_info=True)

test_data = tfds.load('cifar10', split=tfds.Split.TEST)

In [4]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
batch_size = 64
train_num = int(info.splits['train'].num_examples / 10) * 9

train_data = train_data.shuffle(train_num)
train_data = train_data.map(map_func=parse_aug_fn, num_parallel_calls=AUTOTUNE)
train_data = train_data.batch(batch_size).prefetch(buffer_size=AUTOTUNE)

valid_data = valid_data.map(map_func=parse_aug_fn, num_parallel_calls=AUTOTUNE)
valid_data = valid_data.batch(batch_size).prefetch(buffer_size=AUTOTUNE)

test_data = test_data.map(map_func=parse_aug_fn, num_parallel_calls=AUTOTUNE)
test_data = test_data.batch(batch_size).prefetch(buffer_size=AUTOTUNE)

In [5]:
inputs = keras.Input(shape=(32, 32, 3))

x = layers.Conv2D(64, (3, 3))(inputs)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)
x = layers.MaxPool2D()(x)
x = layers.Conv2D(128, (3, 3))(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)
x = layers.Conv2D(256, (3, 3))(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)
x = layers.Conv2D(128, (3, 3))(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)
x = layers.Conv2D(64, (3, 3))(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)
x = layers.Flatten()(x)
x = layers.Dense(64)(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)
x = layers.Dropout(0.5)(x)

outputs = layers.Dense(10, activation='softmax')(x)

model_4 = keras.Model(inputs, outputs, name='model-4')

model_4.summary()

Model: "model-4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 30, 30, 64)        1792      
_________________________________________________________________
batch_normalization (BatchNo (None, 30, 30, 64)        256       
_________________________________________________________________
re_lu (ReLU)                 (None, 30, 30, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 128)       73856     
_________________________________________________________________
batch_normalization_1 (Batch (None, 13, 13, 128)       512 

In [6]:
model_dir = 'lab5-logs\\models\\'

log_dir = os.path.join('lab5-logs', 'run-4-batchnormalization')

model_cbk = keras.callbacks.TensorBoard(log_dir=log_dir)
model_mckp = keras.callbacks.ModelCheckpoint(model_dir + '\\run-4-best-model.h5', monitor='val_categorical_accuracy',
                                             save_best_only=True, mode='max')

In [7]:
model_4.compile(keras.optimizers.Adam(), loss=keras.losses.CategoricalCrossentropy(), metrics=[keras.metrics.CategoricalAccuracy()])

In [9]:
history_1 = model_4.fit(train_data, epochs=100, validation_data=valid_data, callbacks=[model_cbk, model_mckp])

Epoch 1/100
704/704 [==============================] - 74s 105ms/step - loss: 1.5352 - categorical_accuracy: 0.4542 - val_loss: 0.0000e+00 - val_categorical_accuracy: 0.0000e+00
Epoch 2/100
704/704 [==============================] - 99s 141ms/step - loss: 1.3716 - categorical_accuracy: 0.5198 - val_loss: 1.4357 - val_categorical_accuracy: 0.5038
Epoch 3/100
704/704 [==============================] - 117s 166ms/step - loss: 1.2730 - categorical_accuracy: 0.5542 - val_loss: 1.2169 - val_categorical_accuracy: 0.5742
Epoch 4/100
704/704 [==============================] - 118s 167ms/step - loss: 1.1992 - categorical_accuracy: 0.5843 - val_loss: 1.7570 - val_categorical_accuracy: 0.4472
Epoch 5/100
704/704 [==============================] - 120s 171ms/step - loss: 1.1336 - categorical_accuracy: 0.6105 - val_loss: 1.4032 - val_categorical_accuracy: 0.5096
Epoch 6/100
704/704 [==============================] - 122s 173ms/step - loss: 1.0874 - categorical_accuracy: 0.6274 - val_loss: 1.4019 - v

704/704 [==============================] - 157s 224ms/step - loss: 0.5458 - categorical_accuracy: 0.8216 - val_loss: 0.7408 - val_categorical_accuracy: 0.7574
Epoch 49/100
704/704 [==============================] - 148s 210ms/step - loss: 0.5354 - categorical_accuracy: 0.8249 - val_loss: 0.7153 - val_categorical_accuracy: 0.7636
Epoch 50/100
704/704 [==============================] - 121s 172ms/step - loss: 0.5231 - categorical_accuracy: 0.8282 - val_loss: 0.8175 - val_categorical_accuracy: 0.7428
Epoch 51/100
704/704 [==============================] - 121s 172ms/step - loss: 0.5276 - categorical_accuracy: 0.8291 - val_loss: 0.7419 - val_categorical_accuracy: 0.7586
Epoch 52/100
704/704 [==============================] - 121s 172ms/step - loss: 0.5186 - categorical_accuracy: 0.8316 - val_loss: 1.0922 - val_categorical_accuracy: 0.6842
Epoch 53/100
704/704 [==============================] - 121s 171ms/step - loss: 0.5255 - categorical_accuracy: 0.8267 - val_loss: 0.8016 - val_categorica

704/704 [==============================] - 151s 215ms/step - loss: 0.4153 - categorical_accuracy: 0.8651 - val_loss: 0.5971 - val_categorical_accuracy: 0.8038
Epoch 96/100
704/704 [==============================] - 172s 244ms/step - loss: 0.4192 - categorical_accuracy: 0.8651 - val_loss: 0.6394 - val_categorical_accuracy: 0.7950
Epoch 97/100
704/704 [==============================] - 166s 236ms/step - loss: 0.4114 - categorical_accuracy: 0.8677 - val_loss: 0.6381 - val_categorical_accuracy: 0.7962
Epoch 98/100
704/704 [==============================] - 159s 227ms/step - loss: 0.4126 - categorical_accuracy: 0.8652 - val_loss: 0.7294 - val_categorical_accuracy: 0.7712
Epoch 99/100
704/704 [==============================] - 170s 241ms/step - loss: 0.4058 - categorical_accuracy: 0.8707 - val_loss: 0.7249 - val_categorical_accuracy: 0.7652
Epoch 100/100
704/704 [==============================] - 151s 214ms/step - loss: 0.4088 - categorical_accuracy: 0.8678 - val_loss: 0.7188 - val_categoric

In [10]:
loss, acc = model_4.evaluate(test_data)
print('\nModel-4 Accuracy: {}%'.format(acc))

    157/Unknown - 6s 40ms/step - loss: 0.7119 - categorical_accuracy: 0.7764
Model-4 Accuracy: 0.7764000296592712%
